In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

class CompositePeakFWHM:
    def __init__(self):
        """初始化合成峰FWHM分析器"""
        pass
    
    def smooth_curve(self, x, y, method='savgol', **kwargs):
        """
        平滑曲线的多种方法
        
        Parameters:
        -----------
        x : array_like
            x坐标数据
        y : array_like
            y坐标数据（强度）
        method : str
            平滑方法：'savgol', 'gaussian', 'moving_average'
        **kwargs : dict
            平滑参数
        
        Returns:
        --------
        y_smooth : array_like
            平滑后的y数据
        """
        if method == 'savgol':
            window_length = kwargs.get('window_length', min(51, len(y)//4*2+1))  # 确保是奇数
            if window_length >= len(y):
                window_length = len(y)//2*2-1 if len(y)//2*2-1 >= 3 else 3
            polyorder = kwargs.get('polyorder', min(3, window_length-1))
            y_smooth = signal.savgol_filter(y, window_length, polyorder)
            
        elif method == 'gaussian':
            sigma = kwargs.get('sigma', 2.0)
            y_smooth = gaussian_filter1d(y, sigma)
            
        elif method == 'moving_average':
            window = kwargs.get('window', 10)
            y_smooth = np.convolve(y, np.ones(window)/window, mode='same')
            
        else:
            raise ValueError("method must be 'savgol', 'gaussian', or 'moving_average'")
            
        return y_smooth
    
    def estimate_baseline(self, x, y, method='linear_endpoints'):
        """
        估计基线
        
        Parameters:
        -----------
        x : array_like
            x坐标数据
        y : array_like
            y坐标数据
        method : str
            基线估计方法
            
        Returns:
        --------
        baseline : array_like
            基线数据
        """
        if method == 'linear_endpoints':
            # 使用端点线性插值
            n_edge = max(1, len(y) // 20)  # 使用两端各5%的数据点
            left_mean = np.mean(y[:n_edge])
            right_mean = np.mean(y[-n_edge:])
            baseline = np.linspace(left_mean, right_mean, len(y))
            
        elif method == 'polynomial':
            # 多项式拟合基线（排除峰区域）
            # 找到可能的峰区域
            peak_threshold = np.percentile(y, 70)
            mask = y < peak_threshold
            if np.sum(mask) > 10:
                p = np.polyfit(x[mask], y[mask], 2)
                baseline = np.polyval(p, x)
            else:
                # 回退到线性方法
                baseline = self.estimate_baseline(x, y, 'linear_endpoints')
                
        elif method == 'minimum_envelope':
            # 最小包络法
            from scipy.ndimage import minimum_filter1d
            window = len(y) // 10
            baseline = minimum_filter1d(y, size=window, mode='constant')
            
        return baseline
    
    def find_composite_peak_properties(self, x, y, baseline):
        """
        找到合成峰的整体属性
        
        Parameters:
        -----------
        x : array_like
            x坐标数据
        y : array_like
            y坐标数据
        baseline : array_like
            基线数据
            
        Returns:
        --------
        properties : dict
            包含峰的属性：peak_height, peak_x, effective_baseline
        """
        # 去除基线
        y_corrected = y - baseline
        
        # 找到最高点
        max_idx = np.argmax(y_corrected)
        peak_height = y_corrected[max_idx]
        peak_x = x[max_idx]
        
        # 计算有效基线（在峰区域的平均基线）
        # 找到峰的大致范围
        half_max = peak_height / 2
        
        # 从峰顶向两边搜索，找到半高点的大致范围
        left_idx = max_idx
        while left_idx > 0 and y_corrected[left_idx] > half_max:
            left_idx -= 1
            
        right_idx = max_idx
        while right_idx < len(y_corrected) - 1 and y_corrected[right_idx] > half_max:
            right_idx += 1
        
        # 扩展搜索范围
        left_idx = max(0, left_idx - len(y)//20)
        right_idx = min(len(y)-1, right_idx + len(y)//20)
        
        effective_baseline = np.mean(baseline[left_idx:right_idx+1])
        
        properties = {
            'peak_height': peak_height + effective_baseline,
            'peak_x': peak_x,
            'effective_baseline': effective_baseline,
            'corrected_peak_height': peak_height,
            'peak_range': (left_idx, right_idx)
        }
        
        return properties
    
    def find_half_height_positions(self, x, y, peak_properties):
        """
        找到半高位置
        
        Parameters:
        -----------
        x : array_like
            x坐标数据
        y : array_like
            y坐标数据
        peak_properties : dict
            峰属性
            
        Returns:
        --------
        positions : dict
            半高位置信息
        """
        effective_baseline = peak_properties['effective_baseline']
        peak_height = peak_properties['peak_height']
        half_height = effective_baseline + (peak_height - effective_baseline) / 2
        
        # 找到峰的最高点索引
        peak_x = peak_properties['peak_x']
        peak_idx = np.argmin(np.abs(x - peak_x))
        
        # 向左搜索半高点
        left_half_x = None
        for i in range(peak_idx, -1, -1):
            if y[i] <= half_height:
                if i < len(y) - 1:
                    # 线性插值获得精确位置
                    x1, y1 = x[i], y[i]
                    x2, y2 = x[i+1], y[i+1]
                    if y2 != y1:
                        left_half_x = x1 + (half_height - y1) * (x2 - x1) / (y2 - y1)
                    else:
                        left_half_x = x1
                else:
                    left_half_x = x[i]
                break
        
        # 向右搜索半高点
        right_half_x = None
        for i in range(peak_idx, len(y)):
            if y[i] <= half_height:
                if i > 0:
                    # 线性插值获得精确位置
                    x1, y1 = x[i-1], y[i-1]
                    x2, y2 = x[i], y[i]
                    if y2 != y1:
                        right_half_x = x1 + (half_height - y1) * (x2 - x1) / (y2 - y1)
                    else:
                        right_half_x = x2
                else:
                    right_half_x = x[i]
                break
        
        positions = {
            'half_height': half_height,
            'left_half_x': left_half_x,
            'right_half_x': right_half_x,
            'peak_idx': peak_idx
        }
        
        return positions
    
    def calculate_composite_fwhm(self, x, y, smooth_method='savgol', baseline_method='linear_endpoints', 
                                plot=True, **kwargs):
        """
        计算合成峰的FWHM
        
        Parameters:
        -----------
        x : array_like
            x坐标数据
        y : array_like
            y坐标数据
        smooth_method : str
            平滑方法
        baseline_method : str
            基线估计方法
        plot : bool
            是否绘图
        **kwargs : dict
            其他参数
            
        Returns:
        --------
        results : dict
            分析结果
        """
        # 1. 平滑曲线
        y_smooth = self.smooth_curve(x, y, method=smooth_method, **kwargs)
        
        # 2. 估计基线
        baseline = self.estimate_baseline(x, y_smooth, method=baseline_method)
        
        # 3. 找到合成峰属性
        peak_properties = self.find_composite_peak_properties(x, y_smooth, baseline)
        
        # 4. 找到半高位置
        half_positions = self.find_half_height_positions(x, y_smooth, peak_properties)
        
        # 5. 计算FWHM
        if half_positions['left_half_x'] is not None and half_positions['right_half_x'] is not None:
            fwhm = half_positions['right_half_x'] - half_positions['left_half_x']
        else:
            fwhm = None
            print("Warning: Could not find both half-height positions")
        
        results = {
            'original_y': y,
            'smoothed_y': y_smooth,
            'baseline': baseline,
            'peak_properties': peak_properties,
            'half_positions': half_positions,
            'fwhm': fwhm
        }
        
        # 绘图
        if plot:
            self.plot_results(x, results)
        
        return results
    
    def plot_results(self, x, results):
        """绘制分析结果"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # 上图：原始数据、平滑数据和基线
        ax1.plot(x, results['original_y'], 'lightgray', alpha=0.7, label='原始数据')
        ax1.plot(x, results['smoothed_y'], 'b-', linewidth=2, label='平滑数据')
        ax1.plot(x, results['baseline'], 'g--', linewidth=1, label='基线')
        
        # 标记峰顶
        peak_props = results['peak_properties']
        ax1.plot(peak_props['peak_x'], peak_props['peak_height'], 'ro', 
                markersize=8, label=f"峰顶 ({peak_props['peak_x']:.2f}, {peak_props['peak_height']:.2f})")
        
        ax1.set_xlabel('X')
        ax1.set_ylabel('强度')
        ax1.set_title('峰数据分析 - 原始数据与处理')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 下图：FWHM分析
        ax2.plot(x, results['smoothed_y'], 'b-', linewidth=2, label='平滑数据')
        ax2.plot(x, results['baseline'], 'g--', linewidth=1, label='基线')
        
        half_pos = results['half_positions']
        
        # 绘制半高线
        if half_pos['half_height'] is not None:
            ax2.axhline(y=half_pos['half_height'], color='r', linestyle=':', 
                       alpha=0.7, label=f"半高线 (y = {half_pos['half_height']:.2f})")
        
        # 标记半高点
        if half_pos['left_half_x'] is not None:
            ax2.plot(half_pos['left_half_x'], half_pos['half_height'], 'ro', 
                    markersize=6, label=f"左半高点 ({half_pos['left_half_x']:.2f})")
        
        if half_pos['right_half_x'] is not None:
            ax2.plot(half_pos['right_half_x'], half_pos['half_height'], 'ro', 
                    markersize=6, label=f"右半高点 ({half_pos['right_half_x']:.2f})")
        
        # 绘制FWHM
        if results['fwhm'] is not None:
            ax2.annotate('', xy=(half_pos['right_half_x'], half_pos['half_height']), 
                        xytext=(half_pos['left_half_x'], half_pos['half_height']),
                        arrowprops=dict(arrowstyle='<->', color='red', lw=2))
            
            # 添加FWHM文本
            mid_x = (half_pos['left_half_x'] + half_pos['right_half_x']) / 2
            ax2.text(mid_x, half_pos['half_height'] + (ax2.get_ylim()[1] - ax2.get_ylim()[0]) * 0.05,
                    f'FWHM = {results["fwhm"]:.3f}', 
                    ha='center', va='bottom', fontsize=12, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
        
        ax2.set_xlabel('X')
        ax2.set_ylabel('强度')
        ax2.set_title('FWHM分析结果')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# 生成测试数据
def generate_test_data():
    """生成测试用的合成多峰数据"""
    np.random.seed(42)
    x = np.linspace(0, 20, 1000)
    
    # 创建不同类型的测试峰
    test_cases = {}
    
    # 1. 单峰（用于对比）
    single_peak = 10 * np.exp(-0.5 * ((x - 10) / 1.5) ** 2) + 1 + 0.2 * np.random.normal(0, 1, len(x))
    test_cases['单峰'] = (x, single_peak)
    
    # 2. 双峰（部分重叠）
    peak1 = 8 * np.exp(-0.5 * ((x - 8) / 1.2) ** 2)
    peak2 = 6 * np.exp(-0.5 * ((x - 12) / 1.0) ** 2)
    double_peak = peak1 + peak2 + 1.5 + 0.3 * np.random.normal(0, 1, len(x))
    test_cases['双峰'] = (x, double_peak)
    
    # 3. 三峰（高度重叠的合成峰）
    peak1 = 7 * np.exp(-0.5 * ((x - 7) / 1.0) ** 2)
    peak2 = 9 * np.exp(-0.5 * ((x - 10) / 0.8) ** 2)
    peak3 = 5 * np.exp(-0.5 * ((x - 13) / 1.1) ** 2)
    triple_peak = peak1 + peak2 + peak3 + 2 + 0.4 * np.random.normal(0, 1, len(x))
    test_cases['三峰'] = (x, triple_peak)
    
    # 4. 不对称双峰
    peak1 = 12 * np.exp(-0.5 * ((x - 8) / 0.8) ** 2)
    peak2 = 4 * np.exp(-0.5 * ((x - 11.5) / 1.5) ** 2)
    asymmetric_peak = peak1 + peak2 + 1 + 0.25 * np.random.normal(0, 1, len(x))
    test_cases['不对称双峰'] = (x, asymmetric_peak)
    
    return test_cases

# 运行测试
if __name__ == "__main__":
    # 创建分析器
    analyzer = CompositePeakFWHM()
    
    # 生成测试数据
    test_cases = generate_test_data()
    
    print("合成多峰FWHM分析结果:")
    print("=" * 50)
    
    for name, (x, y) in test_cases.items():
        print(f"\n分析 {name}:")
        print("-" * 30)
        
        try:
            results = analyzer.calculate_composite_fwhm(
                x, y, 
                smooth_method='savgol',
                baseline_method='linear_endpoints',
                window_length=31,
                polyorder=3,
                plot=True
            )
            
            if results['fwhm'] is not None:
                print(f"FWHM: {results['fwhm']:.3f}")
                print(f"峰高: {results['peak_properties']['peak_height']:.3f}")
                print(f"峰位: {results['peak_properties']['peak_x']:.3f}")
                print(f"基线: {results['peak_properties']['effective_baseline']:.3f}")
                print(f"左半高点: {results['half_positions']['left_half_x']:.3f}")
                print(f"右半高点: {results['half_positions']['right_half_x']:.3f}")
            else:
                print("无法计算FWHM")
                
        except Exception as e:
            print(f"分析出错: {e}")
    
    print("\n" + "=" * 50)
    print("分析完成！")
    
    # 演示不同平滑方法的效果
    print("\n不同平滑方法对双峰的处理效果对比:")
    x, y = test_cases['双峰']
    
    methods = ['savgol', 'gaussian', 'moving_average']
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, method in enumerate(methods):
        results = analyzer.calculate_composite_fwhm(x, y, smooth_method=method, plot=False)
        
        axes[i].plot(x, y, 'lightgray', alpha=0.7, label='原始数据')
        axes[i].plot(x, results['smoothed_y'], 'b-', linewidth=2, label='平滑数据')
        axes[i].plot(x, results['baseline'], 'g--', label='基线')
        
        if results['fwhm'] is not None:
            half_pos = results['half_positions']
            axes[i].axhline(y=half_pos['half_height'], color='r', linestyle=':', alpha=0.7)
            axes[i].plot([half_pos['left_half_x'], half_pos['right_half_x']], 
                        [half_pos['half_height'], half_pos['half_height']], 'ro-', markersize=4)
            axes[i].set_title(f'{method}\nFWHM = {results["fwhm"]:.3f}')
        else:
            axes[i].set_title(f'{method}\n无法计算FWHM')
        
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

: 